In [2]:
import os 
import sys
import numpy as np
from scipy.spatial.transform import Rotation as R 
import rospy 
import rospkg
import jupyros as jr

from std_msgs.msg import String, Header
from geometry_msgs.msg import PoseStamped, Pose, Point, Quaternion

In [3]:
# add catkin_ws context 
sys.path.append("/home/junting/franka_ws/devel/lib/python3.9/site-packages")

from src.lmp import *
from src.env.multimodal_env import MultiModalEnv
from src.config import cfg_tabletop



Failed to import pyassimp, see https://github.com/ros-planning/moveit/issues/86 for more info
Jupyter environment detected. Enabling Open3D WebVisualizer.
[Open3D INFO] WebRTC GUI backend enabled.
[Open3D INFO] WebRTCWindowSystem: HTTP handshake server disabled.


[nltk_data] Downloading package punkt to /home/junting/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /home/junting/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!


In [4]:
########################################
# initialize environment
########################################
rospy.init_node('jupyter_multimodal', log_level=rospy.DEBUG)
# get package root path 
pkg_root = rospkg.RosPack().get_path('instruct_to_policy')

# setup environment
env = MultiModalEnv(cfg_tabletop)


[DEBUG] [1701956186.621501, 0.000000]: init_node, name[/jupyter_multimodal], pid[2570726]
[DEBUG] [1701956186.622432, 0.000000]: binding to 0.0.0.0 0
[DEBUG] [1701956186.623065, 0.000000]: bound to 0.0.0.0 43491
[DEBUG] [1701956186.623242, 0.000000]: connecting to chao-desktop 43343
[DEBUG] [1701956186.623833, 0.000000]: ... service URL is rosrpc://chao-desktop:43491
[DEBUG] [1701956186.624471, 0.000000]: [/jupyter_multimodal/get_loggers]: new Service instance
[DEBUG] [1701956186.625350, 4126.446000]: ... service URL is rosrpc://chao-desktop:43491
[DEBUG] [1701956186.625841, 4126.447000]: [/jupyter_multimodal/set_logger_level]: new Service instance
[DEBUG] [1701956186.668075, 4126.489000]: connecting to chao-desktop 43343
[DEBUG] [1701956186.668779, 4126.490000]: connecting to chao-desktop 43343
[DEBUG] [1701956186.669837, 4126.491000]: connecting to chao-desktop 57687
[INFO] [1701956186.670903, 4126.492000]: camera_left: Waiting for camera_left/color/camera_info...
[DEBUG] [1701956186

/home/junting/miniforge-pypy3/envs/ros_env/lib/python3.9/site-packages/torch/functional.py:504: UserWarning: torch.meshgrid: in an upcoming release, it will be required to pass the indexing argument. (Triggered internally at ../aten/src/ATen/native/TensorShape.cpp:3190.)
  return _VF.meshgrid(tensors, **kwargs)  # type: ignore[attr-defined]


LANGUAGE BACKBONE USE GRADIENT CHECKPOINTING:  False
[DEBUG] [1701956197.726584, 4137.526000]: Transitioning to ACTIVE (from WAITING_FOR_GOAL_ACK, goal: /jupyter_multimodal-1-4137.526)
[DEBUG] [1701956199.726872, 4139.526000]: Transitioning to WAITING_FOR_RESULT (from ACTIVE, goal: /jupyter_multimodal-1-4137.526)
[DEBUG] [1701956199.727985, 4139.527000]: Transitioning to DONE (from WAITING_FOR_RESULT, goal: /jupyter_multimodal-1-4137.526)
[DEBUG] [1701956217.119438, 4156.908000]: connecting to ('chao-desktop', 43343)
[DEBUG] [1701956217.121959, 4156.911000]: connecting to ('chao-desktop', 43343)
[DEBUG] [1701956217.123618, 4156.912000]: connecting to ('chao-desktop', 43343)
[DEBUG] [1701956217.126968, 4156.916000]: connecting to chao-desktop 43343
[DEBUG] [1701956217.131371, 4156.919000]: connecting to chao-desktop 43343
[DEBUG] [1701956217.134988, 4156.921000]: connecting to chao-desktop 34045
[DEBUG] [1701956217.136669, 4156.922000]: connecting to chao-desktop 34045
[INFO] [170195621

In [26]:
env.reset()

[DEBUG] [1701956502.961148, 4442.501000]: Transitioning to ACTIVE (from WAITING_FOR_GOAL_ACK, goal: /jupyter_multimodal-10-4442.501)
[DEBUG] [1701956504.861817, 4444.401000]: Transitioning to WAITING_FOR_RESULT (from ACTIVE, goal: /jupyter_multimodal-10-4442.501)
[DEBUG] [1701956504.862918, 4444.402000]: Transitioning to DONE (from WAITING_FOR_RESULT, goal: /jupyter_multimodal-10-4442.501)
[DEBUG] [1701956515.873559, 4455.408000]: connecting to ('chao-desktop', 43343)
[DEBUG] [1701956515.875391, 4455.409000]: connecting to ('chao-desktop', 43343)
[DEBUG] [1701956515.876487, 4455.411000]: connecting to ('chao-desktop', 43343)
[DEBUG] [1701956515.877608, 4455.412000]: connecting to chao-desktop 43343
[DEBUG] [1701956515.881477, 4455.415000]: connecting to chao-desktop 43343
[DEBUG] [1701956515.887365, 4455.420000]: connecting to chao-desktop 34045
[DEBUG] [1701956515.889411, 4455.421000]: connecting to chao-desktop 34045
[INFO] [1701956516.933012, 4456.424000]: Environment reset.


In [27]:
print(os.getcwd())

/home/junting/catkin_ws/src/llm-manipulation-bench/instruct_to_policy/scripts


In [28]:
env.detect_objects(object_list=["apple", "plate", "bowl"])

inference time per image: 0.5033056069805752
tensor([[851.1663, 536.1653, 901.8213, 589.9950]]) ['apple']
inference time per image: 0.5018157890008297
tensor([[1117.1006,  429.1297, 1327.5033,  542.5709]]) ['plate']
inference time per image: 0.4937023280072026
tensor([[ 874.4307,  591.7009, 1011.2306,  704.9275]]) ['bowl']
inference time per image: 0.4828629959956743
tensor([[866.6465, 414.9463, 909.0099, 459.0491]]) ['apple']
inference time per image: 0.5025312989891972
tensor([[ 720.2008,  529.7368, 1029.9137,  685.6167]]) ['plate']
inference time per image: 0.49393041798612103
tensor([[674.9136, 380.5056, 756.1622, 453.2097]]) ['bowl']
inference time per image: 0.47305531200254336
tensor([], size=(0, 4)) []
inference time per image: 0.4885192640067544
tensor([], size=(0, 4)) []
inference time per image: 0.47309912301716395
tensor([[1167.4985,  251.1775, 1257.0775,  341.4570]]) ['bowl']
[DEBUG] [1701956522.046382, 4461.524000]: connecting to chao-desktop 34045
Added object apple_0 to

In [29]:
env.scene.visualize_3d_bboxes()
env.get_obj_name_list()

['apple_0', 'plate_0', 'bowl_0']

In [30]:
env.add_scene_objects_to_moveit()

[DEBUG] [1701956524.934074, 4464.413000]: connecting to chao-desktop 34045
Added object apple_0 to planning scene
[DEBUG] [1701956524.967736, 4464.447000]: connecting to chao-desktop 34045
Added object plate_0 to planning scene
[DEBUG] [1701956525.008454, 4464.488000]: connecting to chao-desktop 34045
Added object bowl_0 to planning scene


In [31]:
grasp_pose = env.parse_adaptive_shape_grasp_pose('apple_0')

[INFO] [1701956525.091612, 4464.553000]: Sending perception data to grasp detection service
[DEBUG] [1701956525.097863, 4464.575000]: connecting to chao-desktop 37135


In [32]:
env.planning_scene.get_known_object_names()

['apple_0', 'bowl_0', 'plate_0']

In [33]:
env.objects

{'apple_0': {}, 'plate_0': {}, 'bowl_0': {}}

In [34]:
env.open_gripper()
env.grasp(grasp_pose)


[DEBUG] [1701956527.651042, 4467.126000]: Transitioning to ACTIVE (from WAITING_FOR_GOAL_ACK, goal: /jupyter_multimodal-11-4467.125)
[DEBUG] [1701956527.686720, 4467.159000]: Transitioning to WAITING_FOR_RESULT (from ACTIVE, goal: /jupyter_multimodal-11-4467.125)
[DEBUG] [1701956527.687973, 4467.163000]: Transitioning to DONE (from WAITING_FOR_RESULT, goal: /jupyter_multimodal-11-4467.125)
[DEBUG] [1701956527.689011, 4467.163000]: Waiting for rviz to update


[WARN] [1701956528.744025, 4468.216000]: MoveitEnv: Could not plan cartesian_path to target pose 
position: 
  x: -0.27649660290796935
  y: -0.17286453834982995
  z: 1.1564431072901833
orientation: 
  x: 0.6993529206633681
  y: -0.6677615559119899
  z: 0.1843941981573167
  w: 0.17606469405174707.
 Plan accuracy: 0.6086956521739131


[DEBUG] [1701956538.999824, 4478.468000]: Transitioning to ACTIVE (from WAITING_FOR_GOAL_ACK, goal: /jupyter_multimodal-12-4478.467)
[DEBUG] [1701956539.034604, 4478.503000]: Transitioning to WAITING_FOR_RESULT (from ACTIVE, goal: /jupyter_multimodal-12-4478.467)
[DEBUG] [1701956539.035521, 4478.503000]: Transitioning to DONE (from WAITING_FOR_RESULT, goal: /jupyter_multimodal-12-4478.467)
[DEBUG] [1701956539.037950, 4478.507000]: connecting to chao-desktop 34045
[DEBUG] [1701956539.039816, 4478.509000]: Waiting for rviz to update
[DEBUG] [1701956541.633434, 4481.101000]: connecting to chao-desktop 34045
[DEBUG] [1701956541.635070, 4481.103000]: Waiting for rviz to update
[DEBUG] [1701956543.133993, 4482.601000]: connecting to chao-desktop 34045
[DEBUG] [1701956543.135452, 4482.602000]: Waiting for rviz to update


True

In [35]:
env.close_gripper()
env.attach_object('apple_0')

[DEBUG] [1701956544.655736, 4484.121000]: Transitioning to ACTIVE (from WAITING_FOR_GOAL_ACK, goal: /jupyter_multimodal-13-4484.118)
[DEBUG] [1701956546.588791, 4486.054000]: Transitioning to WAITING_FOR_RESULT (from ACTIVE, goal: /jupyter_multimodal-13-4484.118)
[DEBUG] [1701956546.589734, 4486.055000]: Transitioning to DONE (from WAITING_FOR_RESULT, goal: /jupyter_multimodal-13-4484.118)


True

In [36]:
place_pose = env.parse_place_pose('apple_0', 'plate_0')
env.move_to_pose(place_pose)
env.detach_object('apple_0')

[DEBUG] [1701956546.601246, 4486.065000]: Waiting for rviz to update
